In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/", one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
tf.reset_default_graph()

In [4]:
element_size = 28
time_steps = 28
num_classes = 10
state_size = 128

batch_size = 100

In [5]:
X = tf.placeholder(tf.float32, shape=[None, time_steps, element_size], name="inputs")
y = tf.placeholder(tf.float32, shape=[None, num_classes], name="labels")

In [6]:
rnn_cell = tf.contrib.rnn.BasicRNNCell(state_size)
outputs, states = tf.nn.dynamic_rnn(rnn_cell, X, dtype=tf.float32)
# 'outputs' is a tensor of shape [batch_size, time_steps, hidden_size]

In [7]:
last_rnn_output = outputs[:, -1, :]

In [8]:
Wf = tf.Variable(tf.truncated_normal([state_size, num_classes], mean=0, stddev=0.01))
bf = tf.Variable(tf.truncated_normal([num_classes], mean=0, stddev=0.01))

In [9]:
final_output = tf.matmul(last_rnn_output, Wf) + bf

In [10]:
xentropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=final_output, labels=y))
train = tf.train.AdamOptimizer(0.0001).minimize(xentropy)

In [11]:
correct = tf.equal(tf.argmax(y,1), tf.argmax(final_output,1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [12]:
saver = tf.train.Saver()

n_epochs = 200
train_passes = mnist.train.num_examples // batch_size
test_passes = mnist.test.num_examples // batch_size
final_accuracy = 0

In [13]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        for step in range(train_passes):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            x_batch = x_batch.reshape(batch_size, time_steps, element_size)
            sess.run(train, feed_dict={X: x_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: x_batch, y: y_batch})
        
        cum_acc_test = 0
        for jump in range(test_passes):
            xt_batch, yt_batch = mnist.test.next_batch(batch_size)
            xt_batch = xt_batch.reshape(batch_size, time_steps, element_size)
            cum_acc_test = cum_acc_test + accuracy.eval(feed_dict={X: xt_batch, y: yt_batch})
        acc_test = cum_acc_test/test_passes
        
        print("Epoch {:>2d}".format(epoch), "\tTrain accuracy {:.4f}".format(acc_train), 
                                                  "\tTest accuracy {:.4f}".format(acc_test))
        if acc_test > final_accuracy:
            final_accuracy = acc_test

    save_model = saver.save(sess, "models/my_model_final.ckpt")
    print("\nFinal Accuracy {}".format(final_accuracy))

Epoch  0 	Train accuracy 0.8000 	Test accuracy 0.7510
Epoch  1 	Train accuracy 0.8200 	Test accuracy 0.8297
Epoch  2 	Train accuracy 0.9000 	Test accuracy 0.8750
Epoch  3 	Train accuracy 0.8800 	Test accuracy 0.9021
Epoch  4 	Train accuracy 0.8600 	Test accuracy 0.9109
Epoch  5 	Train accuracy 0.9400 	Test accuracy 0.9199
Epoch  6 	Train accuracy 0.9800 	Test accuracy 0.9258
Epoch  7 	Train accuracy 0.9100 	Test accuracy 0.9304
Epoch  8 	Train accuracy 0.9000 	Test accuracy 0.9271
Epoch  9 	Train accuracy 0.9400 	Test accuracy 0.9336
Epoch 10 	Train accuracy 0.8900 	Test accuracy 0.9367
Epoch 11 	Train accuracy 0.9500 	Test accuracy 0.9340
Epoch 12 	Train accuracy 0.9600 	Test accuracy 0.9296
Epoch 13 	Train accuracy 0.9200 	Test accuracy 0.9454
Epoch 14 	Train accuracy 0.9600 	Test accuracy 0.9440
Epoch 15 	Train accuracy 0.9700 	Test accuracy 0.9469
Epoch 16 	Train accuracy 0.9500 	Test accuracy 0.9471
Epoch 17 	Train accuracy 0.9400 	Test accuracy 0.9492
Epoch 18 	Train accuracy 0.9

Epoch 151 	Train accuracy 1.0000 	Test accuracy 0.9767
Epoch 152 	Train accuracy 1.0000 	Test accuracy 0.9752
Epoch 153 	Train accuracy 0.9900 	Test accuracy 0.9763
Epoch 154 	Train accuracy 0.9900 	Test accuracy 0.9785
Epoch 155 	Train accuracy 0.9900 	Test accuracy 0.9782
Epoch 156 	Train accuracy 1.0000 	Test accuracy 0.9780
Epoch 157 	Train accuracy 1.0000 	Test accuracy 0.9794
Epoch 158 	Train accuracy 1.0000 	Test accuracy 0.9796
Epoch 159 	Train accuracy 0.9900 	Test accuracy 0.9787
Epoch 160 	Train accuracy 1.0000 	Test accuracy 0.9776
Epoch 161 	Train accuracy 1.0000 	Test accuracy 0.9792
Epoch 162 	Train accuracy 1.0000 	Test accuracy 0.9793
Epoch 163 	Train accuracy 1.0000 	Test accuracy 0.9772
Epoch 164 	Train accuracy 0.9700 	Test accuracy 0.9789
Epoch 165 	Train accuracy 1.0000 	Test accuracy 0.9791
Epoch 166 	Train accuracy 1.0000 	Test accuracy 0.9781
Epoch 167 	Train accuracy 1.0000 	Test accuracy 0.9783
Epoch 168 	Train accuracy 1.0000 	Test accuracy 0.9776
Epoch 169 